In [1]:
from database.market import Market
from database.strategy import Strategy
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None
import pytz

In [2]:
market = Market()
strategy = Strategy()

In [3]:
market.connect()
prices = market.retrieve("prices")
market.disconnect()

In [4]:
strategy.connect()
quarterly_sim = strategy.retrieve("quarterly_short_sim")
strategy.disconnect()

In [5]:
quarterly_sim.columns

Index(['_id', 'year', 'quarter', 'ticker',
       'quarterly_price_regression_prediction', 'score', 'model_training_year',
       'category_training_year'],
      dtype='object')

In [6]:
prices["date"] = pd.to_datetime(prices["date"])
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]

In [7]:
prices["date"].max()

Timestamp('2021-10-08 00:00:00+0000', tz='UTC')

In [8]:
final_sim = prices.merge(quarterly_sim,on=["year","quarter","ticker"],how="right")
final_sim["quarterly_delta"] = (final_sim["quarterly_price_regression_prediction"] - final_sim["adjClose"]) / final_sim["adjClose"]
final_sim = final_sim[(final_sim["date"]>=prices["date"].max())]

In [9]:
current_sim[(current_sim["quarter"]==4) & (current_sim["year"]==2021)]

NameError: name 'current_sim' is not defined

In [ ]:
final_sim[final_sim["score"]>0.90].groupby("ticker").mean().sort_values("quarterly_delta",ascending=False).head(20)

In [38]:
trades = []
analysis = []
seats = 11
for quarterly_category_training_year in tqdm(range(1,5)):
    for quarterly_model_training_year in tqdm(range(1,6)):
        for value in [True,False]:
            current_sim = quarterly_sim[(quarterly_sim["model_training_year"]==quarterly_model_training_year) & (quarterly_sim["category_training_year"]==quarterly_category_training_year)]
            final_sim = prices.merge(current_sim,on=["year","quarter","ticker"],how="left")
            final_sim = final_sim[["year","quarter","week","date","ticker","adjClose","quarterly_price_regression_prediction","score"]]
            final_sim["quarterly_delta"] = (final_sim["quarterly_price_regression_prediction"] - final_sim["adjClose"]) / final_sim["adjClose"]
            final_sim = final_sim[(final_sim["year"]>=2019)]
            start = final_sim["date"].min()
            end = final_sim["date"].max()
            date = start
            final_sim.sort_values("date",ascending=True,inplace=True)
            if not value:
                final_sim["quarterly_delta"] = final_sim["quarterly_delta"] * -1
            analysis.append({
                  "qmty":quarterly_model_training_year,
                  "qcty":quarterly_category_training_year,
                 "index_size":final_sim.index.size})
            for delta in range(0,15,5):
                d = float(delta/100)
                for score in range(60,100,10):
                    date = start
                    while date <= end:
                        req = float(score/100)
                        if date.weekday() > 4:
                            date = date + timedelta(days=1)
                        try:
                            todays_sim = final_sim[(final_sim["date"]==date) 
                                                   & (final_sim["quarterly_delta"] >= d) \
                                                  & (final_sim["score"] >= req)].sort_values("quarterly_delta",ascending=False)
                            if todays_sim.index.size < 1:
                                date = date + timedelta(days=1)
                            else:
                                for seat in range(min(seats,todays_sim.index.size)):
                                    try:
                                        trade = todays_sim.iloc[seat]
                                        ticker = trade["ticker"]
                                        buy_price = trade["adjClose"]
                                        exit_price = buy_price * (1+delta)
                                        max_days = min(30,(datetime(date.year,12,24).astimezone(pytz.utc)-date).days)
                                        max_date = date + timedelta(days=max_days)
                                        exits = final_sim[(final_sim["ticker"]==ticker) & 
                                                            (final_sim["date"] > date) & \
                                                          (final_sim["date"] <= max_date) & \
                                                        (final_sim["year"] == date.year) & \
                                                          (final_sim["adjClose"] >= exit_price)]
                                        if exits.index.size < 1:
                                            exits = final_sim[(final_sim["ticker"]==ticker) & \
                                                          (final_sim["year"] == date.year) & \
                                                        (final_sim["date"] >max_date) & \
                                                          (final_sim["adjClose"] >= buy_price)].sort_values("date",ascending=False)
                                            if exits.index.size > 0:
                                                exits.sort_values("date",inplace=True)
                                                sell_trade = exits.iloc[0]
                                            else:
                                                exits = final_sim[(final_sim["ticker"]==ticker) & \
                                                        (final_sim["date"] >max_date) & \
                                                          (final_sim["year"] == date.year)].sort_values("date")
                                                exits.sort_values("date",inplace=True)
                                                sell_trade = exits.iloc[exits.index.size - 1]
                                        else:
                                            exits.sort_values("date",inplace=True)
                                            sell_trade = exits.iloc[0]
                                        trade["sell_date"] = sell_trade["date"]
                                        trade["sell_price"] = sell_trade["adjClose"]
                                        trade["delta"] = (trade["sell_price"] - trade["adjClose"]) / trade["adjClose"]
                                        trade["score"] = score
                                        trade["d"] = delta
                                        trade["qcty"] = quarterly_category_training_year
                                        trade["qmty"] = quarterly_model_training_year
                                        trade["value"] = value
                                        trade["seat"] = seat
                                        strategy.store("quarterly_trades",pd.DataFrame([trade]))
                                        date = sell_trade["date"] + timedelta(days=1)
                                    except Exception as e:
                                        continue
                                date = date + timedelta(days=1)
                        except Exception as e:
                            date = date + timedelta(days=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [33:48<00:00, 507.13s/it]
